# Google Gemma 3 27B Function Calling Example

Google Gemma 3 27B It is an open, multilingual, multimodal Vision-Language model. It handles context windows up to 128k tokens, understands over 140 languages, and offers improved math, reasoning, and chat capabilities, including structured outputs and function calling.

Gemma 3 can be used for agentic workflows and has very strong instruction following capabilities. While there are no dedicated tool/function calling special tokens, you can prompt it to do function calling through careful instruction. Gemma 3 27B is available via [AI Studio](https://aistudio.google.com/prompts/new_chat?model=gemma-3-27b-it) and the Gen AI API. Get an API key from [AI Studio](https://aistudio.google.com/apikey) and use it to make requests to the Gen AI API.

Below is an example of how to use function calling with Gemma 3 27B It. In this example the first user message includes the general instruction how and when to use function calling and example flow. The prompt with the Gemma 3 template would look like this:

1. Send user message with instructions and function definitons and first user message.

```
<bos><start_of_turn>user
At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available, you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response. When using a ```tool_call``` think step by step why and how it should be used.

The following Python methods are available:

\`\`\`python
def convert(amount: float, currency: str, new_currency: str) -> float:
    """Convert the currency with the latest exchange rate

    Args:
      amount: The amount of currency to convert
      currency: The currency to convert from
      new_currency: The currency to convert to
    """
\`\`\`

User: {{user_message}}<end_of_turn>
<start_of_turn>model
```

3. Handle Model response when a tool/function is used.

```
Okay, I need to convert $200,000 to EUR. I will use the `convert` function for this.
\`\`\`tool_code
convert(amount=200000.0, currency="USD", new_currency="EUR")
\`\`\`
```

4. Execute local function and create tool output string

```
\`\`\`tool_output
180000.0
\`\`\`
```

5. Send the tool output as new request to the model

```
<bos><start_of_turn>user
At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available, you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response. When using a ```tool_call``` think step by step why and how it should be used.

The following Python methods are available:

\`\`\`python
def convert(amount: float, currency: str, new_currency: str) -> float:
    """Convert the currency with the latest exchange rate

    Args:
      amount: The amount of currency to convert
      currency: The currency to convert from
      new_currency: The currency to convert to
    """
\`\`\`

User: {{user_message}}<end_of_turn>
<start_of_turn>model
Okay, I need to convert $200,000 to EUR. I will use the `convert` function for this.
\`\`\`tool_code
convert(amount=200000.0, currency="USD", new_currency="EUR")
\`\`\`<end_of_turn>
<start_of_turn>user
\`\`\`tool_output
180000.0
\`\`\`<end_of_turn>
<start_of_turn>model
```

6. Final Response: `$200,000 is equivalent to €180,000. Is there anything else I can help you with?`


Now, let's test this using the GenAI API. If you want to do this locally with, e.g. ollama. You can use the prompts and simulate the function execution.


In [2]:
!pip install google-genai

Then we create our `client`, define Gemma as model id and create a helper method `extract_tool_call`. This method parses the model responses and checks if there is a \`\`\`tool_code\`\`\`. If there is one it uses the `eval` method to run it, extract the result and create a \`\`\`tool_output\`\`\`.

_Note: We use `eval` only for demonstration purposes if you plan to use this in production you should add more security and safety as it will execute model generated code in your environment._

In [3]:
import os
from google import genai
import re
# create client
api_key = os.getenv("GEMINI_API_KEY", "PLACE_YOUR_API_KEY_HERE" ) #
client = genai.Client(api_key=api_key)

# speicfy the model id
model_id = "gemma-3-27b-it"

# extract the tool call from the response
def extract_tool_call(text):
    import io
    from contextlib import redirect_stdout

    pattern = r"```tool_code\s*(.*?)\s*```"
    match = re.search(pattern, text, re.DOTALL)
    if match:
        code = match.group(1).strip()
        # Capture stdout in a string buffer
        f = io.StringIO()
        with redirect_stdout(f):
            result = eval(code)
        output = f.getvalue()
        r = result if output == '' else output
        return f'```tool_output\n{str(r).strip()}\n```'''
    return None


Next, we define a simple example for a function we want to use. Here is is a `convert` method that simulates the conversion calcuation of currencies. Since we use `eval` the method we want to use for function calling needs to be available in the environment.

We define our first user prompt including our instructions and function signature with a docstring and args and a template string for our user message.

In [19]:
def convert(amount: float, currency: str, new_currency: str) -> float:
  # demo implementation
  return amount * 7.23


def get_exchange_rate(currency: str, new_currency: str) -> float:
    # demo implementation
    return 0.14

instruction_prompt_with_function_calling = '''At each turn, if you decide to invoke any of the function(s), it should be wrapped with ```tool_code```. The python methods described below are imported and available, you can only use defined methods. The generated code should be readable and efficient. The response to a method will be wrapped in ```tool_output``` use it to call more tools or generate a helpful, friendly response. When using a ```tool_call``` think step by step why and how it should be used.

The following Python methods are available:

```python
def convert(amount: float, currency: str, new_currency: str) -> float:
    """Convert the currency with the latest exchange rate

    Args:
      amount: The amount of currency to convert
      currency: The currency to convert from
      new_currency: The currency to convert to
    """

def get_exchange_rate(currency: str, new_currency: str) -> float:
    """Get the latest exchange rate for the currency pair

    Args:
      currency: The currency to convert from
      new_currency: The currency to convert to
    """
```

User: {user_message}
'''

The `genai` SDK supports stateful chat session, which makes it quite easy to test our example as we can easily append the different messages. First, we start with a simple greeting to see what Gemma does.

In [20]:
chat = client.chats.create(model=model_id)

response = chat.send_message(instruction_prompt_with_function_calling.format(user_message="hello"))
print(response.text)

Hello! How can I help you today? Do you need a currency conversion or an exchange rate?



Nice! it greeted us back and didn't use any function call. Okay now lets ask it to convert some currency.

In [21]:
response = chat.send_message("What is $200,000 in RMB?")
print(response.text)

Okay, I need to convert $200,000 USD to Chinese Yuan (RMB). I will use the `convert` function for this.

```tool_code
convert(amount=200000.0, currency="USD", new_currency="RMB")
```


Great! it generated out ```tool_code```, which we can now use and extract and to cool our method.

In [22]:
call_response = extract_tool_call(response.text)
print(call_response)

```tool_output
1446000.0
```


After we have the response from our tool call we send a final message to generate a user friendly output.

In [23]:
response = chat.send_message(call_response)
print(response.text)

$200,000 USD is equivalent to 1,446,000.0 RMB. Is there anything else I can help you with?

